In [21]:
import datetime
import ipywidgets as widgets
from IPython.display import display
from backend import API

key_dict = {}
keys = []
with open("api_keys.txt", 'r') as fp:
    for i, line in enumerate(fp.readlines()[1:]):
        arr = line.strip().split(',')
        key_dict[i] = ','.join(arr)
        keys.append((arr[0], i))

default_key_secret = key_dict[keys[0][1]].split(',')
account_output = widgets.Output()
output_log = widgets.Output()

with account_output:
    api = API(default_key_secret[1], default_key_secret[2])

In [22]:
w_symbol = widgets.Text(
    # value='',
    placeholder='如BTC,后台会自动补齐以满足不同品种格式',
    description='交易币种:',
    disabled=False
)
w_amount = widgets.FloatText(
    # value='',
    placeholder='',
    description='交易金额/U:',
    disabled=False   
)
w_quantity = widgets.FloatText(
    # value='',
    placeholder='',
    description='数量/张数:',
    disabled=False   
)
w_api_key = widgets.Text(
    value=default_key_secret[1],
    # placeholder='输入',
    description='API_KEY:',
    disabled=False
)
w_secret = widgets.Text(
    value=default_key_secret[2],
    # placeholder='输入',
    description='SECRET:',
    disabled=False
)

w_mode = widgets.Dropdown(
    options=[('实盘', '0'), ('模拟', '1')],
    value='0',
    description='交易模式:',
)

# 查询账户总资产
def query_balance():
    total_balance, funding_balance, union_balance = api.query_wallet_balance()
    _, valid_assets = api.query_pm_account_asset()
    account_output.clear_output()
    with account_output:
        print('账户总资产: {:.2f}U, 统一账户总资产: {:.2f}U, 资金账户总资产:{:.2f}' \
              .format(total_balance, union_balance, funding_balance))
        print('合约持仓情况')
        for d in valid_assets:
            if float(d['cmWalletBalance']) > 0:
                print('{} cm合约净资产: {}, 杠杆净资产: {}'. \
                      format(d['asset'], d['cmWalletBalance'], d['crossMarginFree']))

w_key = widgets.Dropdown(
    options=keys,
    value=0,
    description='api_key名称:',
)
def on_key_name_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        key_id = change['new']
        values = key_dict[key_id].split(',')
        w_api_key.value = values[1]
        w_secret.value = values[2]
        api.set_key(values[1], values[2])
        query_balance()
w_key.observe(on_key_name_change)

In [23]:
layout_flex = widgets.Layout(width='auto', height='auto')

w_btn_open_order = widgets.Button(
    description='开仓',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_flex,
)
def click_w_btn_open_order(b):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    ccy = w_symbol.value
    amount = w_amount.value
    with output_log:
        print(t, ", 开仓: ", ccy, ', 金额: ', amount)
        api.open(ccy, amount)
w_btn_open_order.on_click(click_w_btn_open_order)

w_btn_clear_order = widgets.Button(
    description='平仓',
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_flex,
)
def click_w_btn_clear_order(b):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    ccy = w_symbol.value
    amount = w_amount.value
    with output_log:
        print(t, ", 平仓: ", ccy, ', 金额: ', amount)
        api.close(ccy, amount)
w_btn_clear_order.on_click(click_w_btn_clear_order)


w_btn_margin_buy = widgets.Button(
    description='杠杆下单做多,优先使用数量',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_flex,
)
def click_w_btn_margin_buy(b):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    ccy = w_symbol.value
    amount = w_amount.value
    quantity = w_quantity.value
    with output_log:
        print(t, ", 杠杆下单: ", ccy, ', 金额: ', amount, ', 数量: ', quantity)
        api.margin_trade(ccy, amount, 'BUY', quantity)
w_btn_margin_buy.on_click(click_w_btn_margin_buy)

w_btn_margin_sell = widgets.Button(
    description='杠杆下单做空,优先使用数量',
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_flex,
)
def click_w_btn_margin_sell(b):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    ccy = w_symbol.value
    amount = w_amount.value
    quantity = w_quantity.value
    with output_log:
        print(t, ", 杠杆下单: ", ccy, ', 金额: ', amount, ', 数量: ', quantity)
        api.margin_trade(ccy, amount, 'SELL', quantity)
w_btn_margin_sell.on_click(click_w_btn_margin_sell)


w_btn_cm_buy = widgets.Button(
    description='币本位合约下单做多,数量代表合约张数',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_flex,
)
def click_w_btn_cm_buy(b):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    ccy = w_symbol.value
    quantity = w_quantity.value
    with output_log:
        print(t, ", 杠杆下单: ", ccy, ', 数量: ', quantity)
        api.cm_trade(ccy, 'BUY', quantity)
w_btn_cm_buy.on_click(click_w_btn_cm_buy)

w_btn_cm_sell = widgets.Button(
    description='币本位合约下单做空,数量代表合约张数',
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_flex,
)
def click_w_btn_cm_sell(b):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    ccy = w_symbol.value
    quantity = w_quantity.value
    with output_log:
        print(t, ", 杠杆下单: ", ccy, ', 数量: ', quantity)
        api.cm_trade(ccy, 'SELL', quantity)
w_btn_cm_sell.on_click(click_w_btn_cm_sell)


trans_types = '''MAIN_FUNDING 现货钱包转向资金钱包
FUNDING_MAIN 资金钱包转向现货钱包
MAIN_PORTFOLIO_MARGIN 现货钱包转向统一账户钱包
PORTFOLIO_MARGIN_MAIN 统一账户钱包转向现货钱包'''
# FUNDING_PORTFOLIO_MARGIN 资金钱包转向统一账户钱包
# PORTFOLIO_MARGIN_FUNDING 统一账户钱包转向资金钱包'''

trans_type_ids = []
trans_type_dict = {}
for i, trans_type in enumerate(trans_types.splitlines()):
    t_type, t_name = trans_type.split()
    trans_type_ids.append((t_name, i))
    trans_type_dict[i] = t_type

w_transfer_key = widgets.Dropdown(
    options=trans_type_ids,
    value=0,
    description='划转类型:',
)
w_btn_transfer = widgets.Button(
    description='万向划转',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_flex,
)
def click_w_btn_transfer(b):
    t = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    tranfer_type = trans_type_dict[w_transfer_key.value]
    ccy = w_symbol.value
    quantity = w_quantity.value
    with output_log:
        print('{}, {}, {}'.format(tranfer_type, ccy, quantity))
        if w_transfer_key.value >= 0 and w_transfer_key.value <= 3:
            api.transfer(tranfer_type, ccy, quantity)
w_btn_transfer.on_click(click_w_btn_transfer)


w_btn_clean = widgets.Button(
    description='清空日志',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    layout=layout_flex,
)
def click_w_btn_clean(b):
    output_log.clear_output()
w_btn_clean.on_click(click_w_btn_clean)

In [24]:
from ipywidgets import HBox, VBox
from ipywidgets import GridspecLayout

grid = GridspecLayout(7, 3)
grid[0,:] = HBox([w_key, w_api_key, w_secret])
grid[1,:] = HBox([w_symbol, w_quantity, w_amount])
grid[2,:] = HBox([w_btn_open_order, w_btn_clear_order])
grid[3,:] = HBox([w_btn_margin_buy, w_btn_margin_sell])
grid[4,:] = HBox([w_btn_cm_buy, w_btn_cm_sell])
grid[5,:] = HBox([w_btn_transfer, w_transfer_key])
grid[6,:] = HBox([w_btn_clean])
grid

GridspecLayout(children=(HBox(children=(Dropdown(description='api_key名称:', options=(('default', 0), ('test1', …

In [25]:
display(account_output)

Output()

In [26]:
display(output_log)

Output()